In [7]:
import pandas as pd
import re

def process_text(input_text):
    # Добавляем пробелы к букве "R", если она идет слитно с цифрой
    pattern1 = r'(R)(\d+)'
    processed_text = re.sub(pattern1, r'R \2', input_text)

    pattern2 = r'(\d+)(R)'
    processed_text = re.sub(pattern2, r'\1 R', processed_text)

    pattern3 = r'(\/)(\d+)'
    processed_text = re.sub(pattern3, r'/ \2', processed_text)

    pattern4 = r'(\d+)(\/)'
    processed_text = re.sub(pattern4, r'\1 /', processed_text)

    pattern5 = r'(\\)(\d+)'
    processed_text = re.sub(pattern5, r'\\ \2', processed_text)

    pattern6 = r'(\d+)(\\)'
    processed_text = re.sub(pattern6, r'\1 \\', processed_text)

    # Заменяем символы "/", "\", "|" на пробелы
    # processed_text = re.sub(r'[\/\\|]', ' ', processed_text)

    # Заменяем символы "/" на " / "
    # processed_text = processed_text.replace("/", " / ")
    # Заменяем символы "\" на " \ "
    # processed_text = processed_text.replace("\\", " \\ ")
    # Заменяем символы "|" на " | "
    processed_text = processed_text.replace("|", " | ")

    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text

def create_bio_tags(row):
    name = row['offer']
    width = str(row['width'])
    height = str(row['height'])
    radius = str(row['radius'])
    brand = row['brand']
    line = str(row['line'])
    
    # Создаем пустой список для хранения тегов
    tags = ['O'] * len(name.split())
    Begin_line = True
    # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
    for i, word in enumerate(name.split()):
        if word == '/':
            continue
        if word == width:
            tags[i] = 'B-width'
        elif word == height:
            tags[i] = 'B-height'
        elif word == radius:
            tags[i] = 'B-radius'
        elif word in brand:
            tags[i] = 'B-brand'
        elif word in line:
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
            
    # Собираем теги в строку
    return ','.join(tags)

def extract_brands(text):
    brands = re.findall(r'Bridgestone|BRIDGESTONE|Бриджстоун|bridgestone|Бриджстон|Bridgeston|бриджстоун|бриджстон', text)
    return brands

Наши данные

In [8]:
pth = '/home/sondors/Documents/price/Разметка для Игоря Bridgestone.xlsx'
clms = ['XL',
        'RFT']
    
df = pd.read_excel(pth, dtype=str)
df = df.drop(columns=clms)
df = df.rename(columns={'Ширина': 'width', 'Радиус': 'radius', 'Высота': 'height','PRICE_NAME': 'offer', 'Линейка': 'line', 'Инд. Скорост': 'v_ind'})
df = df[df['offer'].notna()]
# print(df.keys())
# df

df['brand'] = df['offer'].apply(extract_brands)
df['offer'] = df['offer'].apply(process_text)
df['type'] = ['наши_данные' for i in range(len(df['offer']))]
# Применяем функцию к каждой строке DataFrame и создаем новую колонку с тегами
df['BIO_Tags'] = df.apply(create_bio_tags, axis=1)
df

,offer,width,height,radius,v_ind,line,brand,type,BIO_Tags
0,13 / 175 / 70 Bridgestone Ice Cruiser 7000S 82T ш,175,70,17,82S,Ice Cruiser 7000S,[Bridgestone],наши_данные,"O,O,B-width,O,B-height,B-brand,B-line,I-line,I..."
1,14 / 155 / 65 Bridgestone Blizzak LM005 79T XL,155,65,14,79T,Blizzak LM005,[Bridgestone],наши_данные,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
2,14 / 165 / 65 Bridgestone Blizzak LM005 79T,165,65,16,79T,Blizzak LM005,[Bridgestone],наши_данные,"O,O,B-width,O,B-height,B-brand,B-line,I-line,O"
3,14 / 165 / 65 Bridgestone Ecopia EP150 79S,165,65,16,79S,Ecopia EP150,[Bridgestone],наши_данные,"O,O,B-width,O,B-height,B-brand,B-line,I-line,O"
4,14 / 165 / 70 Bridgestone Turanza T005 81T,165,70,16,81T,Turanza T005,[Bridgestone],наши_данные,"O,O,B-width,O,B-height,B-brand,B-line,I-line,O"
...,...,...,...,...,...,...,...,...,...
8273,R 19 225 / 45 92W Bridgestone Dueler H/P Sport...,225,45,19,92W,Dueler H/P Sport,"[Bridgestone, BRIDGESTONE]",наши_данные,"O,B-radius,B-width,O,B-height,O,B-brand,B-line..."
8274,R 19 225 / 45 92W Bridgestone Dueler H/P Sport...,225,45,19,92W,Dueler H/P Sport,"[Bridgestone, BRIDGESTONE]",наши_данные,"O,B-radius,B-width,O,B-height,O,B-brand,B-line..."
8275,R 20 155 / 60 80Q Bridgestone Ecopia EP500 * B...,155,60,20,80Q,Ecopia EP500,"[Bridgestone, BRIDGESTONE]",наши_данные,"O,B-radius,B-width,O,B-height,O,B-brand,B-line..."
8276,"Бриджстоун 2155017 S 91 REVO-GZ, BRIDGESTONE 1...",215,50,17,91S,Blizzak Revo-GZ,"[Бриджстоун, BRIDGESTONE]",наши_данные,"B-brand,O,O,O,O,B-brand,O,O,O"


In [9]:
ilocc = df.iloc[8273]
print(ilocc.offer, '\n')
BIO = ilocc.BIO_Tags.split(',')
ofer = ilocc.offer.split()
for i in range(len(BIO)):
    print(f"{ofer[i]} --> {BIO[i]}")


R 19 225 / 45 92W Bridgestone Dueler H/P Sport (уценка) BRIDGESTONE 9776 

R --> O
19 --> B-radius
225 --> B-width
/ --> O
45 --> B-height
92W --> O
Bridgestone --> B-brand
Dueler --> B-line
H/P --> I-line
Sport --> I-line
(уценка) --> O
BRIDGESTONE --> B-brand
9776 --> O


Опенсорс

In [44]:
def clear_df(df):
    df = df[df['offer'].notna()]
    df = df[df['brand'].notna()]
    df = df[df['width'].notna()]
    df = df[df['height'].notna()]
    df = df[df['radius'].notna()]
    df = df[df.apply(lambda row: str(row['width']) in row['offer'], axis=1)]
    df = df[df.apply(lambda row: str(row['height']) in row['offer'], axis=1)]
    df = df[df.apply(lambda row: str(row['radius']) in row['offer'], axis=1)]
    return df.reset_index(drop=True)
    
pth = '/home/sondors/Documents/price/BERT_NER/opensource.csv'
clms = ['width_indices', 'height_indices', 'radius_indices', 'brand_indices',
       'line_indices']
    
df_opensource = pd.read_csv(pth, sep=';')
df_opensource = df_opensource.drop(columns=clms)

df_opensource = clear_df(df_opensource)
df_opensource['offer'] = df_opensource['offer'].apply(process_text)

# Применяем функцию к каждой строке DataFrame и создаем новую колонку с тегами
df_opensource['BIO_Tags'] = df_opensource.apply(create_bio_tags, axis=1)

df_opensource


,width,height,radius,brand,offer,type,line,BIO_Tags
0,175,65,14,Amtel,Amtel 175 / 65 R 14 82Q Nordmaster (В-228),Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O"
1,175,70,13,Avatyre,Avatyre 175 / 70 R 13 82Q Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
2,205,55,16,Avatyre,Avatyre 205 / 55 R 16 91T Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
3,215,65,16,Avatyre,Avatyre 215 / 65 R 16 98T Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
4,215,65,16,Avatyre,Avatyre 215 / 65 R 16 98T Freeze Товар с уценкой,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O,O"
...,...,...,...,...,...,...,...,...
5865,425,85,21,НК.ШЗ,425 / 85 R 21 КАМА-1260-1 НК.ШЗ 14 146 J,грузовая_резина,КАМА-1260-1,"B-width,O,B-height,O,B-radius,B-line,B-brand,O..."
5866,425,85,21,НК.ШЗ,425 / 85 R 21 КАМА-1260-2 НК.ШЗ 18,грузовая_резина,КАМА-1260-2,"B-width,O,B-height,O,B-radius,B-line,B-brand,O"
5867,425,85,21,Омск.ШЗ,425 / 85 R 21 О-184 TYREX CRG POWER Омск.ШЗ 14...,грузовая_резина,О-184 TYREX CRG POWER,"B-width,O,B-height,B-line,B-radius,I-line,I-li..."
5868,425,85,21,Омск.ШЗ,425 / 85 R 21 О-184 TYREX CRG POWER Омск.ШЗ 18...,грузовая_резина,О-184 TYREX CRG POWER,"B-width,O,B-height,B-line,B-radius,I-line,I-li..."


In [45]:
ilocc.radius

'14'

In [48]:
ilocc = df_opensource.iloc[5867]
print(ilocc.offer, '\n')
BIO = ilocc.BIO_Tags.split(',')
ofer = ilocc.offer.split()
for i in range(len(BIO)):
    print(f"{ofer[i]} --> {BIO[i]}")


425 / 85 R 21 О-184 TYREX CRG POWER Омск.ШЗ 14 146 K 

425 --> B-width
/ --> O
85 --> B-height
R --> B-line
21 --> B-radius
О-184 --> I-line
TYREX --> I-line
CRG --> I-line
POWER --> I-line
Омск.ШЗ --> B-brand
14 --> O
146 --> O
K --> O


In [12]:
# single_value_brand_df.to_csv('/home/sondors/Documents/price/Разметка_1.csv', sep=';',index=False)